In [ ]:
# --- 1° Passo para o TCC Configuração e Importação de Bibliotecas ---

# Instalação de bibliotecas (necessário no Google Colab, pode pular se já tiver instalado localmente)
!pip install pandas scikit-learn joblib lightgbm openpyxl matplotlib seaborn

# Importação das bibliotecas
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier # Algoritmo de exemplo: RandomForest
# from lightgbm import LGBMClassifier # Descomente e use se preferir LightGBM
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, confusion_matrix
import joblib # Para salvar e carregar o pipeline

# Para visualização da matriz de confusão
import matplotlib.pyplot as plt
import seaborn as sns

# Para upload e leitura de .xlsx no Colab
from google.colab import files
import io
import os # Para lidar com caminhos de arquivo

In [ ]:
# --- 2° Passo para o TCC Upload do XLSX e Carregamento dos Dados ---

print("Por favor, faça o upload do seu arquivo XLSX (treinoML.xlsx):")
uploaded = files.upload()

# Obtém o nome do primeiro (e geralmente único) arquivo enviado
# Substitua 'relatorio_bi_certificados_04-07-2025.xlsx' pelo nome do seu arquivo XLSX se for diferente
# Ou use file_name = list(uploaded.keys())[0] para pegar o nome automaticamente
XLSX_FILE_NAME = 'treinoML.xlsx' # Ajuste este nome para o seu arquivo, se necessário

# Carregar os dados do arquivo XLSX
print(f"\nCarregando dados de '{XLSX_FILE_NAME}'...")
try:
    df = pd.read_excel(io.BytesIO(uploaded[XLSX_FILE_NAME]), engine='openpyxl')
    print("Dados carregados com sucesso!")
    print(f"Shape dos dados: {df.shape}")
    print("Primeiras 5 linhas:")
    print(df.head())
    print("\nInformações sobre as colunas:")
    df.info()
except KeyError:
    print(f"ERRO: O arquivo '{XLSX_FILE_NAME}' não foi encontrado entre os arquivos enviados.")
    print("Verifique se o nome do arquivo no código (XLSX_FILE_NAME) corresponde ao nome do arquivo que você fez upload.")
    exit()
except Exception as e:
    print(f"ERRO ao carregar o arquivo XLSX: {e}")
    exit()

In [ ]:
# --- 3° Passo para o TCC Definição de Features, Alvo e Tratamento de Nulos --
# --- Configurações de Colunas para o Python ---
TARGET_COLUMN = 'SLA_Atrasado_3Dias'

NUMERICAL_FEATURES = [
.
    # ADICIONE AQUI TODAS AS SUAS OUTRAS COLUNAS NUMÉRICAS
]

CATEGORICAL_FEATURES = [
.
.
    # ADICIONE AQUI TODAS AS SUAS OUTRAS COLUNAS DE TEXTO
]

# --- Verificações e Preparação de X (Features) e y (Alvo) ---
if TARGET_COLUMN not in df.columns:
    raise ValueError(f"A coluna alvo '{TARGET_COLUMN}' não foi encontrada no DataFrame. Verifique o nome da coluna no seu XLSX.")

# Converte a coluna alvo para int, preenchendo nulos com 0 (isso foi feito no PQ, mas reconfirma)
# No entanto, para TREINAMENTO, queremos que os 'null's no alvo sejam removidos!
# Vamos converter para float primeiro para aceitar NaN temporariamente.
df[TARGET_COLUMN] = df[TARGET_COLUMN].fillna(-1) # Placeholder temporário para nulos
df[TARGET_COLUMN] = df[TARGET_COLUMN].astype(int)

# --- REMOVER LINHAS ONDE A COLUNA ALVO É NULA (tickets abertos) PARA O TREINAMENTO ---
# O modelo APENAS aprenderá com tickets que foram CONCLUÍDOS e cujo SLA é conhecido (0 ou 1).
initial_rows = df.shape[0]
df_train = df[df[TARGET_COLUMN] != -1].copy() # Filtra as linhas com placeholder
removed_rows = initial_rows - df_train.shape[0]
print(f"\nRemovidas {removed_rows} linhas do treinamento (tickets em aberto ou com dados incompletos para o alvo).")

if df_train.empty:
    raise ValueError("Não há dados suficientes para treinar o modelo após remover linhas com alvo nulo.")

X = df_train[NUMERICAL_FEATURES + CATEGORICAL_FEATURES].copy()
y = df_train[TARGET_COLUMN].copy()


# --- Tratamento de Valores Nulos nas FEATURES (IMPORTANTE!) ---
print("\nVerificando e tratando valores nulos nas FEATURES para o TREINAMENTO:")
for col in NUMERICAL_FEATURES:
    if X[col].isnull().any():
        median_val = X[col].median()
        X[col] = X[col].fillna(median_val)
        print(f"  Preenchidos nulos na coluna numérica '{col}' com a mediana ({median_val:.2f}).")

for col in CATEGORICAL_FEATURES:
    if X[col].isnull().any():
        X[col] = X[col].fillna('Desconhecido')
        print(f"  Preenchidos nulos na coluna categórica '{col}' com 'Desconhecido'.")

print("\nStatus de nulos nas FEATURES após tratamento (espera-se 0 nulos):")
print(X.isnull().sum())
if X.isnull().sum().sum() > 0:
    print("AVISO: Ainda existem valores nulos nas features após o tratamento. Verifique os dados.")

In [ ]:
# ---  4° Passo para o TCC Criação do Pipeline de Pré-processamento e Modelo ---

preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', NUMERICAL_FEATURES),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), CATEGORICAL_FEATURES)
    ],
    remainder='drop'
)

classifier = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
# Opcional: Para LightGBM
# import lightgbm as lgb
# classifier = lgb.LGBMClassifier(random_state=42, class_weight='balanced')

model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', classifier)
])

print("\nPipeline de ML criado com sucesso!")
print(model_pipeline)

In [ ]:
# --- 5° Passo para o TCC Divisão dos Dados e Treinamento do Pipeline ---

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print(f"\nDados divididos: Treino={X_train.shape}, Teste={X_test.shape}")

print("\nIniciando treinamento do modelo...")
model_pipeline.fit(X_train, y_train)
print("Treinamento concluído!")

In [ ]:
# --- 6° Passo para o TCC Avaliação do Modelo ---

print("\nAvaliação do Modelo no Conjunto de Teste:")
y_pred = model_pipeline.predict(X_test)
y_proba = model_pipeline.predict_proba(X_test)[:, 1]

print(f"Acurácia: {accuracy_score(y_test, y_pred):.4f}")
print(f"AUC (Area Under the Curve): {roc_auc_score(y_test, y_proba):.4f}")
print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred))

In [ ]:
# ---  Matriz de Confusão Visual ---

cm = confusion_matrix(y_test, y_pred)

class_names = ['Dentro do SLA (0)', 'Atrasado (1)']

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=class_names, yticklabels=class_names)

plt.title('Matriz de Confusão - Previsão de Atraso de SLA')
plt.ylabel('Valores Reais')
plt.xlabel('Previsões do Modelo')

# Anotações para TCC
# TN: Verdadeiro Negativo (Previu 0, Real 0) - Acertou que NÃO atrasaria
# FP: Falso Positivo (Previu 1, Real 0) - Errou, previu que atrasaria, mas NÃO atrasou (alarme falso)
# FN: Falso Negativo (Previu 0, Real 1) - Errou, previu que NÃO atrasaria, mas ATRASOU (o pior para SLA!)
# TP: Verdadeiro Positivo (Previu 1, Real 1) - Acertou que atrasaria

# Ajuste as coordenadas x,y para posicionar os textos na sua plotagem
# Exemplo: plt.text(0.5, 1.2, 'FP (Falso Positivo)', transform=plt.gca().transAxes, horizontalalignment='center', color='red')
# plt.text(0.5, -0.2, 'Verdadeiros Positivos (TP)', ha='center', va='center', fontsize=10, color='green', transform=plt.gca().transAxes)
# plt.text(-0.2, 0.5, 'Falsos Positivos (FP)', ha='center', va='center', fontsize=10, color='red', transform=plt.gca().transAxes, rotation=90)

plt.tight_layout()
plt.show()

print(f"\nDetalhes da Matriz de Confusão:")
print(f"Verdadeiros Negativos (TN): {cm[0,0]} (Previu 0, Real 0)")
print(f"Falsos Positivos (FP): {cm[0,1]} (Previu 1, Real 0)")
print(f"Falsos Negativos (FN): {cm[1,0]} (Previu 0, Real 1)")
print(f"Verdadeiros Positivos (TP): {cm[1,1]} (Previu 1, Real 1)")

In [ ]:
# --- 7° Passo para o TCC Salvar o Pipeline Treinado e Baixar para a Máquina Local ---

# Nome do arquivo do modelo a ser salvo no ambiente Colab (e depois baixado)
MODEL_SAVE_NAME = 'ML_certificado_V2.pkl'

# Salvar o pipeline completo
joblib.dump(model_pipeline, MODEL_SAVE_NAME)
print(f"\nPipeline de ML salvo com sucesso no ambiente Colab como: {MODEL_SAVE_NAME}")

# Baixar o arquivo do modelo para sua máquina local
print(f"Fazendo download de '{MODEL_SAVE_NAME}' para sua máquina local...")
files.download(MODEL_SAVE_NAME)
print("Download concluído!")

In [ ]:
# --- Gráficos de Curva ROC e Precision-Recall ---

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score

# As variáveis y_test e y_proba já foram calculadas na Célula 6
# y_test: Valores reais (0 ou 1) do conjunto de teste
# y_proba: Probabilidades previstas pelo modelo para a classe 1 (atrasado)

# --- 1. Curva ROC (Receiver Operating Characteristic) ---
print("\nGerando Curva ROC...")
fpr, tpr, thresholds_roc = roc_curve(y_test, y_proba)
roc_auc = auc(fpr, tpr) # Já calculamos na Célula 6, mas podemos refazer aqui

plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'Curva ROC (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Classificador Aleatório')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Taxa de Falsos Positivos (FPR)') # 1 - Especificidade
plt.ylabel('Taxa de Verdadeiros Positivos (TPR) / Sensibilidade') # Recall
plt.title('Curva ROC - Previsão de Atraso de SLA')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

print(f"Interpretação da Curva ROC:")
print(f"- A Curva ROC mostra a taxa de verdadeiros positivos (TPR) versus a taxa de falsos positivos (FPR) em vários limiares de classificação.")
print(f"- A Área sob a Curva (AUC) de {roc_auc:.4f} é uma medida da capacidade do modelo de distinguir entre as classes. Um valor próximo de 1.0 indica um excelente poder discriminatório.")
print(f"- Quanto mais a curva se afasta da linha diagonal (classificador aleatório), melhor o modelo.")

# --- 2. Curva Precision-Recall ---
print("\nGerando Curva Precision-Recall...")
precision, recall, thresholds_pr = precision_recall_curve(y_test, y_proba)
avg_precision = average_precision_score(y_test, y_proba)

plt.figure(figsize=(10, 6))
plt.plot(recall, precision, color='blue', lw=2, label=f'Curva Precision-Recall (AP = {avg_precision:.4f})')
plt.xlabel('Recall (Sensibilidade)')
plt.ylabel('Precisão')
plt.title('Curva Precision-Recall - Previsão de Atraso de SLA')
plt.legend(loc='lower left')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.grid(True)
plt.show()

print(f"Interpretação da Curva Precision-Recall:")
print(f"- A Curva Precision-Recall exibe a precisão e o recall do modelo em diferentes limiares. É especialmente útil quando há um desbalanceamento de classes, onde a classe positiva (atraso) é menos frequente.")
print(f"- A Média de Precisão (AP) de {avg_precision:.4f} é a área sob a Curva Precision-Recall. Quanto maior o AP, melhor o modelo em identificar a classe positiva sem muitos falsos positivos.")
print(f"- Um valor alto tanto para precisão quanto para recall ao longo da curva indica um modelo robusto.")

# --- Opcional: Gráfico de Probabilidade de Previsão vs Limiar (Threshold) ---
# Este gráfico pode ajudar a escolher um limite de corte ideal para sua operação.
# Exemplo: onde Precision e Recall se cruzam, ou onde há um bom equilíbrio.
# Pode ser muito útil para o TCC para justificar a escolha do limite de 0.5 (ou outro).

# F1-score para diferentes thresholds
f1_scores = 2 * (precision * recall) / (precision + recall)
f1_scores[f1_scores == float('inf')] = 0 # Trata divisão por zero (ocorre para recall=0)

# Encontrar o melhor threshold para F1-score (ou para um equilíbrio específico)
best_f1_threshold = thresholds_pr[f1_scores.argmax()]
best_f1_score = f1_scores.max()
print(f"\nMelhor Threshold para F1-score: {best_f1_threshold:.4f} (F1-score: {best_f1_score:.4f})")


plt.figure(figsize=(10, 6))
plt.plot(thresholds_pr, precision[:-1], label='Precisão') # precision[:-1] porque thresholds_pr tem um item a menos
plt.plot(thresholds_pr, recall[:-1], label='Recall')
plt.plot(thresholds_pr, f1_scores[:-1], label='F1-score', linestyle='--')
plt.axvline(x=0.5, color='gray', linestyle=':', label='Threshold Padrão (0.5)')
plt.axvline(x=best_f1_threshold, color='green', linestyle='--', label=f'Melhor F1 Threshold ({best_f1_threshold:.2f})')

plt.xlabel('Limiar de Probabilidade (Threshold)')
plt.ylabel('Métrica')
plt.title('Precisão, Recall e F1-score vs. Limiar de Probabilidade')
plt.legend(loc='best')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.grid(True)
plt.show()

print(f"Interpretação da Escolha do Limiar (Threshold):")
print(f"- O gráfico mostra como a Precisão, o Recall e o F1-score variam conforme o limiar de probabilidade para classificar um ticket como 'Atrasado (1)'.")
print(f"- Um limiar de 0.5 é comumente usado por padrão, onde probabilidades > 0.5 são classificadas como '1'.")
print(f"- A escolha do limiar ideal depende do custo dos erros: se evitar um 'Falso Negativo' (perder um atraso real) é mais crítico, pode-se escolher um limiar menor para aumentar o Recall. Se evitar um 'Falso Positivo' (alarme falso) é mais importante, pode-se escolher um limiar maior para aumentar a Precisão.")